<a href="https://colab.research.google.com/github/FarukKaramanVektorel/yz/blob/master/fineTunning_02_03_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
import torch
import pandas as pd
from google.colab import files
from datasets import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSeq2SeqLM

# CSV dosyasını yükle
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name)

# Sütun isimlerini soru-cevap formatına uygun hale getir
data = data.rename(columns={"Input (Soru)": "question", "Output (Cevap)": "answer"})

# Eğer bağlam (context) sütunu yoksa, boş olarak ekleyin
if "context" not in data.columns:
    data["context"] = ""

# Veri setini Hugging Face formatına çevir
dataset = Dataset.from_pandas(data)

# Model ve tokenizer'ı yükle
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Modeli uygun cihaza taşı
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Eğitim için kullanılan cihaz: {device}")

# Veri ön işleme fonksiyonu
def preprocess_function(examples):
    inputs = [f"question: {q} context: {c}" for q, c in zip(examples["question"], examples["context"])]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)

    labels = tokenizer(text_target=examples["answer"], padding="max_length", truncation=True, max_length=512)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Veri kümesini tokenleştir
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Eğitim argümanlarını belirle
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,  # Hızlandırma için float16 kullan
    push_to_hub=True,
    hub_model_id="abuzerkomurcu78/tst2",
    report_to="none"  # W&B'yi devre dışı bırak
)

# Trainer'ı oluştur
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# Modeli eğit
trainer.train()

# Modeli Hugging Face'e yükle
trainer.push_to_hub()


Saving qa_pairs_all_variations.csv to qa_pairs_all_variations (3).csv
Eğitim için kullanılan cihaz: cuda


Map:   0%|          | 0/453 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-ecbaa67892a2>:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,1.263326
2,No log,1.207971
3,No log,1.165269
4,No log,1.147646
5,No log,1.142521


CommitInfo(commit_url='https://huggingface.co/abuzerkomurcu78/tst2/commit/5588ee8dd9a39d5eae030fa1d4e1c7eeaf8ec145', commit_message='End of training', commit_description='', oid='5588ee8dd9a39d5eae030fa1d4e1c7eeaf8ec145', pr_url=None, repo_url=RepoUrl('https://huggingface.co/abuzerkomurcu78/tst2', endpoint='https://huggingface.co', repo_type='model', repo_id='abuzerkomurcu78/tst2'), pr_revision=None, pr_num=None)

In [ ]:
# Eğitim argümanlarını belirle
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,  # Hızlandırma için float16 kullan
    push_to_hub=True,
    hub_model_id="abuzerkomurcu78/tst2",
    report_to="none"  # W&B'yi devre dışı bırak
)

# Trainer'ı oluştur
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# Modeli eğit
trainer.train()

# Modeli Hugging Face'e yükle
trainer.push_to_hub()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-fd525ce3fb6f>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
